In [1]:
import pandas as pd
import numpy as np
import tensorflow
import theano
import pickle

from pandas import *

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals.joblib import parallel_backend

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../data/df_for_model.csv')
# df['start_time'] = df['start_time']//1000
df = df[df['result'] != 0.5]
numeric_predictors = ['elo', 'opp_elo', 'diff', 'color',
                      'game_time', 'start_time', 'weekday', 'day']
X = df[numeric_predictors]
y = np.array(df['result'])

In [3]:
X.shape

(2024, 8)

In [4]:
y.shape

(2024,)

In [5]:
X_train = X[:1620]
y_train = y[:1620]
X_test = X[1620:]
y_test = y[1620:]

In [6]:
X_train.shape

(1620, 8)

In [7]:
y_train.shape

(1620,)

In [8]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [9]:
std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [81]:
def _classifier():
    classifier = Sequential()
    classifier.add(Dense(units=8, kernel_initializer='uniform',
                         activation='relu', input_dim=8))
    classifier.add(
        Dense(units=6, kernel_initializer='uniform', activation='softsign'))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])

    return classifier

In [49]:
classifier = _classifier()

In [50]:
classifier.fit(X_train, y_train, batch_size=5, epochs=200)

Epoch 1/200
1620/1620 [==============================] - 16s 10ms/step - loss: 0.6257 - acc: 0.6574
Epoch 2/200
1620/1620 [==============================] - 3s 2ms/step - loss: 0.5659 - acc: 0.7117
Epoch 3/200
1620/1620 [==============================] - 3s 2ms/step - loss: 0.5631 - acc: 0.7086
Epoch 4/200
1620/1620 [==============================] - 3s 2ms/step - loss: 0.5608 - acc: 0.7117
Epoch 5/200
1620/1620 [==============================] - 3s 2ms/step - loss: 0.5596 - acc: 0.7130
Epoch 6/200
1620/1620 [==============================] - 3s 2ms/step - loss: 0.5579 - acc: 0.7167
Epoch 7/200
1620/1620 [==============================] - 3s 2ms/step - loss: 0.5572 - acc: 0.7148
Epoch 8/200
1620/1620 [==============================] - 3s 2ms/step - loss: 0.5567 - acc: 0.7148
Epoch 9/200
1620/1620 [==============================] - 3s 2ms/step - loss: 0.5550 - acc: 0.7179
Epoch 10/200
1620/1620 [==============================] - 3s 2ms/step - loss: 0.5554 - acc: 0.7142
Epoch 11/200
1620

1620/1620 [==============================] - 2s 1ms/step - loss: 0.5233 - acc: 0.7364
Epoch 166/200
1620/1620 [==============================] - 2s 1ms/step - loss: 0.5241 - acc: 0.7377
Epoch 167/200
1620/1620 [==============================] - 2s 1ms/step - loss: 0.5236 - acc: 0.7377
Epoch 168/200
1620/1620 [==============================] - 3s 2ms/step - loss: 0.5223 - acc: 0.7426
Epoch 169/200
1620/1620 [==============================] - 3s 2ms/step - loss: 0.5239 - acc: 0.7340
Epoch 170/200
1620/1620 [==============================] - 2s 1ms/step - loss: 0.5237 - acc: 0.7346
Epoch 171/200
1620/1620 [==============================] - 3s 2ms/step - loss: 0.5235 - acc: 0.7364
Epoch 172/200
1620/1620 [==============================] - 3s 2ms/step - loss: 0.5229 - acc: 0.7395
Epoch 173/200
1620/1620 [==============================] - 4s 2ms/step - loss: 0.5232 - acc: 0.7333
Epoch 174/200
1620/1620 [==============================] - 4s 3ms/step - loss: 0.5227 - acc: 0.7401
Epoch 175/200


In [51]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [82]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[142,  63],
       [ 61, 138]])

In [83]:
(142+138)/(142+63+61+138)*100

69.3069306930693

In [87]:
classifier = KerasClassifier(
    build_fn=_classifier, batch_size=9, epochs=108)

In [88]:
accus = cross_val_score(estimator=classifier, X=X_train,
                        y=y_train, cv=10, n_jobs=-4)

Epoch 1/108
1458/1458 [==============================] - 15s 10ms/step - loss: 0.6535 - acc: 0.6495
Epoch 2/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5705 - acc: 0.7174
Epoch 3/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5636 - acc: 0.7202
Epoch 4/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5607 - acc: 0.7188
Epoch 5/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5592 - acc: 0.7181
Epoch 6/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5588 - acc: 0.7202
Epoch 7/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5576 - acc: 0.7222
Epoch 8/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5560 - acc: 0.7222
Epoch 9/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5567 - acc: 0.7174
Epoch 10/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5559 - acc: 0.7209
Epoch 11/108
1458

1458/1458 [==============================] - 1s 873us/step - loss: 0.5428 - acc: 0.7339
Epoch 56/108
1458/1458 [==============================] - 1s 846us/step - loss: 0.5425 - acc: 0.7270
Epoch 57/108
1458/1458 [==============================] - 1s 866us/step - loss: 0.5419 - acc: 0.7311
Epoch 58/108
1458/1458 [==============================] - 1s 850us/step - loss: 0.5414 - acc: 0.7373
Epoch 59/108
1458/1458 [==============================] - 1s 847us/step - loss: 0.5422 - acc: 0.7346
Epoch 60/108
1458/1458 [==============================] - 1s 845us/step - loss: 0.5422 - acc: 0.7318
Epoch 61/108
1458/1458 [==============================] - 1s 869us/step - loss: 0.5410 - acc: 0.7373
Epoch 62/108
1458/1458 [==============================] - 1s 862us/step - loss: 0.5424 - acc: 0.7277
Epoch 63/108
1458/1458 [==============================] - 1s 854us/step - loss: 0.5407 - acc: 0.7346
Epoch 64/108
1458/1458 [==============================] - 1s 842us/step - loss: 0.5410 - acc: 0.7298
Epo

162/162 [==============================] - 6s 34ms/step
Epoch 1/108
1458/1458 [==============================] - 14s 10ms/step - loss: 0.6507 - acc: 0.6385
Epoch 2/108
1458/1458 [==============================] - 1s 879us/step - loss: 0.5674 - acc: 0.7058
Epoch 3/108
1458/1458 [==============================] - 1s 877us/step - loss: 0.5589 - acc: 0.7126
Epoch 4/108
1458/1458 [==============================] - 1s 864us/step - loss: 0.5551 - acc: 0.7160
Epoch 5/108
1458/1458 [==============================] - 1s 875us/step - loss: 0.5533 - acc: 0.7202
Epoch 6/108
1458/1458 [==============================] - 1s 866us/step - loss: 0.5517 - acc: 0.7167
Epoch 7/108
1458/1458 [==============================] - 1s 867us/step - loss: 0.5491 - acc: 0.7236
Epoch 8/108
1458/1458 [==============================] - 1s 852us/step - loss: 0.5488 - acc: 0.7222
Epoch 9/108
1458/1458 [==============================] - 1s 886us/step - loss: 0.5466 - acc: 0.7181
Epoch 10/108
1458/1458 [====================

1458/1458 [==============================] - 1s 846us/step - loss: 0.5385 - acc: 0.7202
Epoch 53/108
1458/1458 [==============================] - 1s 875us/step - loss: 0.5373 - acc: 0.7291
Epoch 54/108
1458/1458 [==============================] - 1s 858us/step - loss: 0.5368 - acc: 0.7257
Epoch 55/108
1458/1458 [==============================] - 1s 856us/step - loss: 0.5368 - acc: 0.7291
Epoch 56/108
1458/1458 [==============================] - 1s 861us/step - loss: 0.5367 - acc: 0.7257
Epoch 57/108
1458/1458 [==============================] - 1s 854us/step - loss: 0.5361 - acc: 0.7270
Epoch 58/108
1458/1458 [==============================] - 1s 854us/step - loss: 0.5360 - acc: 0.7298
Epoch 59/108
1458/1458 [==============================] - 1s 854us/step - loss: 0.5363 - acc: 0.7339
Epoch 60/108
1458/1458 [==============================] - 1s 854us/step - loss: 0.5358 - acc: 0.7257
Epoch 61/108
1458/1458 [==============================] - 1s 859us/step - loss: 0.5358 - acc: 0.7305
Epo

1458/1458 [==============================] - 2s 1ms/step - loss: 0.5397 - acc: 0.7366
Epoch 107/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5395 - acc: 0.7359
Epoch 108/108
162/162 [==============================] - 6s 37ms/step
Epoch 1/108
1458/1458 [==============================] - 16s 11ms/step - loss: 0.6524 - acc: 0.6516
Epoch 2/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5734 - acc: 0.7092
Epoch 3/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5658 - acc: 0.7133
Epoch 4/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5649 - acc: 0.7106
Epoch 5/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5628 - acc: 0.7167
Epoch 6/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5617 - acc: 0.7099
Epoch 7/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5618 - acc: 0.7085
Epoch 8/108
1458/1458 [==============================] -

1458/1458 [==============================] - 2s 1ms/step - loss: 0.5409 - acc: 0.7387
Epoch 55/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5406 - acc: 0.7401
Epoch 56/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5403 - acc: 0.7401
Epoch 57/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5408 - acc: 0.7339
Epoch 58/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5403 - acc: 0.7407
Epoch 59/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5396 - acc: 0.7421
Epoch 60/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5400 - acc: 0.7407
Epoch 61/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5404 - acc: 0.7339
Epoch 62/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5391 - acc: 0.7387
Epoch 63/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5398 - acc: 0.7346
Epoch 64/108
1458/1458 

1458/1458 [==============================] - 2s 1ms/step - loss: 0.5724 - acc: 0.7112
Epoch 3/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5630 - acc: 0.7160
Epoch 4/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5610 - acc: 0.7167
Epoch 5/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5580 - acc: 0.7209
Epoch 6/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5567 - acc: 0.7126
Epoch 7/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5551 - acc: 0.7099
Epoch 8/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5536 - acc: 0.7188
Epoch 9/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5537 - acc: 0.7188
Epoch 10/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5523 - acc: 0.7174
Epoch 11/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5512 - acc: 0.7209
Epoch 12/108
1458/1458 [======

Epoch 85/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5335 - acc: 0.7407
Epoch 86/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5332 - acc: 0.7325
Epoch 87/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5332 - acc: 0.7394
Epoch 88/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5336 - acc: 0.7325
Epoch 89/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5328 - acc: 0.7318
Epoch 90/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5337 - acc: 0.7318
Epoch 91/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5323 - acc: 0.7366
Epoch 92/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5331 - acc: 0.7366
Epoch 93/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5320 - acc: 0.7387
Epoch 94/108
1458/1458 [==============================] - 2s 1ms/step - loss: 0.5334 - acc: 0.7298
Epoch 95/1

In [89]:
accus

array([0.70370372, 0.72839508, 0.70987656, 0.70370372, 0.6851852 ,
       0.72222224, 0.70987656, 0.69753088, 0.7160494 , 0.6851852 ])

In [90]:
accu_mean = accus.mean()
accu_mean

0.7061728544533253

In [91]:
accu_var = accus.std()
accu_var

0.013580245592377558

In [10]:
def dropout_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=11, kernel_initializer='uniform',
                         activation='relu', input_dim=8))
    classifier.add(Dropout(rate=0.12813308004))
    classifier.add(
        Dense(units=8, kernel_initializer='uniform', activation='softsign'))
    classifier.add(Dropout(rate=0.05108273694))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [11]:
classifier = KerasClassifier(build_fn=dropout_classifier)

In [12]:
parameters = {'batch_size': [9, 18, 27], 'nb_epoch': [100, 250, 500],
              'optimizer': ['rmsprop', 'sgd', 'Nadam', 'Adadelta']}

In [13]:
grid_search = GridSearchCV(classifier, parameters,
                           scoring='accuracy', cv=10, n_jobs=-4)

In [14]:
grid_search = grid_search.fit(X=X_train, y=y_train)

Epoch 1/1
1458/1458 [==============================] - 1s 656us/step - loss: 0.6859 - acc: 0.6296
Epoch 1/1
1458/1458 [==============================] - 1s 612us/step - loss: 0.6875 - acc: 0.6125
Epoch 1/1
1458/1458 [==============================] - 1s 618us/step - loss: 0.6873 - acc: 0.6283
Epoch 1/1
1458/1458 [==============================] - 1s 677us/step - loss: 0.6862 - acc: 0.5905
Epoch 1/1
1458/1458 [==============================] - 1s 692us/step - loss: 0.6872 - acc: 0.6125
Epoch 1/1
1458/1458 [==============================] - 1s 695us/step - loss: 0.6857 - acc: 0.6063
Epoch 1/1
1458/1458 [==============================] - 1s 771us/step - loss: 0.6856 - acc: 0.6070
Epoch 1/1
1458/1458 [==============================] - 1s 736us/step - loss: 0.6870 - acc: 0.6255
Epoch 1/1
1458/1458 [==============================] - 1s 762us/step - loss: 0.6875 - acc: 0.6166
Epoch 1/1
1458/1458 [==============================] - 1s 793us/step - loss: 0.6860 - acc: 0.5741
Epoch 1/1
1458/1458 

1458/1458 [==============================] - 3s 2ms/step - loss: 0.6844 - acc: 0.6283
Epoch 1/1
1458/1458 [==============================] - 3s 2ms/step - loss: 0.6864 - acc: 0.6091
Epoch 1/1
1458/1458 [==============================] - 3s 2ms/step - loss: 0.6852 - acc: 0.6084
Epoch 1/1
1458/1458 [==============================] - 3s 2ms/step - loss: 0.6856 - acc: 0.5912
Epoch 1/1
1458/1458 [==============================] - 3s 2ms/step - loss: 0.6873 - acc: 0.5796
Epoch 1/1
1458/1458 [==============================] - 3s 2ms/step - loss: 0.6874 - acc: 0.5789
Epoch 1/1
1458/1458 [==============================] - 3s 2ms/step - loss: 0.6933 - acc: 0.4739
Epoch 1/1
1458/1458 [==============================] - 3s 2ms/step - loss: 0.6932 - acc: 0.5062
Epoch 1/1
1458/1458 [==============================] - 3s 2ms/step - loss: 0.6931 - acc: 0.5103
Epoch 1/1
1458/1458 [==============================] - 3s 2ms/step - loss: 0.6932 - acc: 0.4835
Epoch 1/1
1458/1458 [=============================

1458/1458 [==============================] - 6s 4ms/step - loss: 0.6900 - acc: 0.6001
Epoch 1/1
1458/1458 [==============================] - 6s 4ms/step - loss: 0.6932 - acc: 0.4966
Epoch 1/1
1458/1458 [==============================] - 6s 4ms/step - loss: 0.6932 - acc: 0.5062
Epoch 1/1
1458/1458 [==============================] - 6s 4ms/step - loss: 0.6931 - acc: 0.5082
Epoch 1/1
1458/1458 [==============================] - 6s 4ms/step - loss: 0.6932 - acc: 0.4938
Epoch 1/1
1458/1458 [==============================] - 6s 4ms/step - loss: 0.6933 - acc: 0.4664
Epoch 1/1
1458/1458 [==============================] - 6s 4ms/step - loss: 0.6932 - acc: 0.4945
Epoch 1/1
1458/1458 [==============================] - 6s 4ms/step - loss: 0.6932 - acc: 0.5027
Epoch 1/1
1458/1458 [==============================] - 6s 4ms/step - loss: 0.6932 - acc: 0.4678
Epoch 1/1
1458/1458 [==============================] - 6s 4ms/step - loss: 0.6932 - acc: 0.4815
Epoch 1/1
1458/1458 [=============================

1458/1458 [==============================] - 8s 6ms/step - loss: 0.6932 - acc: 0.4877
Epoch 1/1
1458/1458 [==============================] - 8s 6ms/step - loss: 0.6932 - acc: 0.5021
Epoch 1/1
1458/1458 [==============================] - 8s 6ms/step - loss: 0.6932 - acc: 0.5007
Epoch 1/1
1458/1458 [==============================] - 8s 6ms/step - loss: 0.6932 - acc: 0.4979
Epoch 1/1
1458/1458 [==============================] - 8s 6ms/step - loss: 0.6932 - acc: 0.5048
Epoch 1/1
1458/1458 [==============================] - 8s 6ms/step - loss: 0.6932 - acc: 0.4890
Epoch 1/1
1458/1458 [==============================] - 9s 6ms/step - loss: 0.6891 - acc: 0.5645
Epoch 1/1
1458/1458 [==============================] - 9s 6ms/step - loss: 0.6852 - acc: 0.5988
Epoch 1/1
1458/1458 [==============================] - 9s 6ms/step - loss: 0.6874 - acc: 0.5782
Epoch 1/1
1458/1458 [==============================] - 9s 6ms/step - loss: 0.6856 - acc: 0.5892
Epoch 1/1
1458/1458 [=============================

1458/1458 [==============================] - 12s 8ms/step - loss: 0.6932 - acc: 0.5000
Epoch 1/1
1458/1458 [==============================] - 12s 8ms/step - loss: 0.6932 - acc: 0.4808
Epoch 1/1
1458/1458 [==============================] - 12s 8ms/step - loss: 0.6844 - acc: 0.6262
Epoch 1/1
1458/1458 [==============================] - 12s 8ms/step - loss: 0.6853 - acc: 0.6008
Epoch 1/1
1458/1458 [==============================] - 12s 8ms/step - loss: 0.6864 - acc: 0.6077
Epoch 1/1
1458/1458 [==============================] - 12s 8ms/step - loss: 0.6853 - acc: 0.6036
Epoch 1/1
1458/1458 [==============================] - 12s 8ms/step - loss: 0.6890 - acc: 0.5974
Epoch 1/1
1458/1458 [==============================] - 12s 8ms/step - loss: 0.6858 - acc: 0.6015
Epoch 1/1
1458/1458 [==============================] - 12s 8ms/step - loss: 0.6840 - acc: 0.6248
Epoch 1/1
1458/1458 [==============================] - 12s 8ms/step - loss: 0.6870 - acc: 0.6084
Epoch 1/1
1458/1458 [===================

In [15]:
best_params = grid_search.best_params_
best_params

{'batch_size': 9, 'nb_epoch': 500, 'optimizer': 'Nadam'}

In [16]:
best_accu = grid_search.best_score_
best_accu

0.7111111111111111

In [34]:
grid_search

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f481293d6a0>,
       fit_params=None, iid='warn', n_jobs=-4,
       param_grid={'batch_size': [9, 18, 27], 'nb_epoch': [100, 250, 500], 'optimizer': ['rmsprop', 'sgd', 'Nadam', 'Adadelta']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [58]:
y_pred = grid_search.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[119,  86],
       [ 33, 166]])

In [39]:
(119+166)/(119+86+33+166)

0.7054455445544554